In [12]:
%matplotlib inline
import matplotlib.pyplot as plt

In [13]:
import numpy as np
import numpy.random as npr
import spglib as spg
import ase
from ase.io import read,write
from ase.spacegroup import crystal
from ase.visualize import view
import cPickle as pck
import pandas as pd
from tqdm import tqdm_notebook

from quippy.potential import Potential, Minim
from ase.optimize import  FIRE
from ase.constraints import UnitCellFilter

In [14]:
from generate_and_relax_structures import generate_crystal_step_1,generate_crystal_step_1_wrapper
from libs.utils import unskewCell,ase2qp,qp2ase,get_standard_frame
from libs.input_structure import input2crystal,getCellParam
from libs.LJ_pressure import make_LJ_input,LJ_vcrelax
from libs.raw_data import z2symb,z2VdWradius,z2Covalentradius,SG2BravaisLattice,WyckTable

In [15]:
import sys
sys.path.insert(0,'/local/git/glosim2/')
from libmatch.soap import get_Soaps
from libmatch.utils import get_soapSize,get_spkit,get_spkitMax,ase2qp,qp2ase
from libmatch.chemical_kernel import deltaKernel,PartialKernels
from GlobalSimilarity import get_environmentalKernels,get_globalKernel

In [16]:
d2r = np.pi/180.

In [17]:
def compare(frame1,frame2):
    centerweight  = 1.
    gaussian_width = 0.1
    cutoff = 3.5
    cutoff_transition_width = 0.5
    nmax = 8
    lmax = 6
    nocenters = []
    is_fast_average = False

    soap_params = {
              'centerweight': centerweight, 
              'gaussian_width': gaussian_width,'cutoff': cutoff, 
              'cutoff_transition_width': cutoff_transition_width,
              'nmax': nmax, 'lmax': lmax, 'is_fast_average':is_fast_average,
              'chem_channels': True ,'nocenters': nocenters,'dispbar':True,
                   }
    ff = []
    for frame in [frame1,frame2]:
        ff.append(ase2qp(frame))
     
    envk = get_environmentalKernels(ff,nthreads=1, nprocess=1, nchunks=1,**soap_params)
    gkern = get_globalKernel(envk, kernel_type='average', zeta=4, gamma=1.0, eps=1e-06, 
                             nthreads=8, normalize_global_kernel=True)
    
    return gkern[0,1]

# info

In [7]:
fileNames = {}
infoPath = './reference_info/'
structurePath = './structures/'
fileNames['crystals'] = structurePath + 'input_crystals_sg1-230-18-10-17.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [8]:
# with open(fileNames['crystals'],'rb') as f:
#     crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

# make some info dict

In [32]:
ElemTable.head()

,covr,elneg,mass,sym,val,vdwr,z
0,0.32,2.20,1.007940,H,1,1.20,1
1,0.46,0.00,4.002602,He,0,1.43,2
2,1.33,0.98,6.941000,Li,1,2.12,3
3,1.02,1.57,9.012182,Be,2,1.98,4
4,0.85,2.04,10.811000,B,3,1.91,5


In [41]:
ElemTable[['z','sym']].values

array([[1, 'H'],
       [2, 'He'],
       [3, 'Li'],
       [4, 'Be'],
       [5, 'B'],
       [6, 'C'],
       [7, 'N'],
       [8, 'O'],
       [9, 'F'],
       [10, 'Ne'],
       [11, 'Na'],
       [12, 'Mg'],
       [13, 'Al'],
       [14, 'Si'],
       [15, 'P'],
       [16, 'S'],
       [17, 'Cl'],
       [18, 'Ar'],
       [19, 'K'],
       [20, 'Ca'],
       [21, 'Sc'],
       [22, 'Ti'],
       [23, 'V'],
       [24, 'Cr'],
       [25, 'Mn'],
       [26, 'Fe'],
       [27, 'Co'],
       [28, 'Ni'],
       [29, 'Cu'],
       [30, 'Zn'],
       [31, 'Ga'],
       [32, 'Ge'],
       [33, 'As'],
       [34, 'Se'],
       [35, 'Br'],
       [36, 'Kr'],
       [37, 'Rb'],
       [38, 'Sr'],
       [39, 'Y'],
       [40, 'Zr'],
       [41, 'Nb'],
       [42, 'Mo'],
       [43, 'Tc'],
       [44, 'Ru'],
       [45, 'Rh'],
       [46, 'Pd'],
       [47, 'Ag'],
       [48, 'Cd'],
       [49, 'In'],
       [50, 'Sn'],
       [51, 'Sb'],
       [52, 'Te'],
       [53, 'I'],
       [54, 'X

In [42]:
z2symb = {z:symb for z,symb in ElemTable[['z','sym']].values}
print z2symb

{1: 'H', 2: 'He', 3: 'Li', 4: 'Be', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 10: 'Ne', 11: 'Na', 12: 'Mg', 13: 'Al', 14: 'Si', 15: 'P', 16: 'S', 17: 'Cl', 18: 'Ar', 19: 'K', 20: 'Ca', 21: 'Sc', 22: 'Ti', 23: 'V', 24: 'Cr', 25: 'Mn', 26: 'Fe', 27: 'Co', 28: 'Ni', 29: 'Cu', 30: 'Zn', 31: 'Ga', 32: 'Ge', 33: 'As', 34: 'Se', 35: 'Br', 36: 'Kr', 37: 'Rb', 38: 'Sr', 39: 'Y', 40: 'Zr', 41: 'Nb', 42: 'Mo', 43: 'Tc', 44: 'Ru', 45: 'Rh', 46: 'Pd', 47: 'Ag', 48: 'Cd', 49: 'In', 50: 'Sn', 51: 'Sb', 52: 'Te', 53: 'I', 54: 'Xe', 55: 'Cs', 56: 'Ba', 57: 'La', 58: 'Ce', 59: 'Pr', 60: 'Nd', 62: 'Sm', 63: 'Eu', 64: 'Gd', 65: 'Tb', 66: 'Dy', 67: 'Ho', 68: 'Er', 69: 'Tm', 70: 'Yb', 71: 'Lu', 72: 'Hf', 73: 'Ta', 74: 'W', 75: 'Re', 76: 'Os', 77: 'Ir', 78: 'Pt', 79: 'Au', 80: 'Hg', 81: 'Tl', 82: 'Pb', 83: 'Bi'}


In [58]:
from mendeleev import element

In [56]:
el = element(14)
print el.vdw_radius*100

210.0


In [63]:
z2VdWradius = {}
for z in ElemTable[['z']].values:
    try:
        z2VdWradius[z[0]] = element(z[0]).vdw_radius/100.
    except:
        print z[0]

In [64]:
print z2VdWradius

{1: 1.1, 2: 1.4, 3: 1.82, 4: 1.53, 5: 1.92, 6: 1.7, 7: 1.55, 8: 1.52, 9: 1.47, 10: 1.54, 11: 2.27, 12: 1.73, 13: 1.84, 14: 2.1, 15: 1.8, 16: 1.8, 17: 1.75, 18: 1.88, 19: 2.75, 20: 2.31, 21: 2.15, 22: 2.11, 23: 2.07, 24: 2.06, 25: 2.05, 26: 2.04, 27: 2.0, 28: 1.97, 29: 1.96, 30: 2.01, 31: 1.87, 32: 2.11, 33: 1.85, 34: 1.9, 35: 1.85, 36: 2.02, 37: 3.03, 38: 2.49, 39: 2.32, 40: 2.23, 41: 2.18, 42: 2.17, 43: 2.16, 44: 2.13, 45: 2.1, 46: 2.1, 47: 2.11, 48: 2.18, 49: 1.93, 50: 2.17, 51: 2.06, 52: 2.06, 53: 1.98, 54: 2.16, 55: 3.43, 56: 2.68, 57: 2.43, 58: 2.42, 59: 2.4, 60: 2.39, 62: 2.36, 63: 2.35, 64: 2.34, 65: 2.33, 66: 2.31, 67: 2.3, 68: 2.29, 69: 2.27, 70: 2.26, 71: 2.24, 72: 2.23, 73: 2.22, 74: 2.18, 75: 2.16, 76: 2.16, 77: 2.13, 78: 2.13, 79: 2.14, 80: 2.23, 81: 1.96, 82: 2.02, 83: 2.07}


In [65]:
z2Covalentradius = {}
for z in ElemTable[['z']].values:
    try:
        z2Covalentradius[z[0]] = element(z[0]).covalent_radius_pyykko/100.
    except:
        print z[0]

In [66]:
print z2Covalentradius

{1: 0.32, 2: 0.46, 3: 1.33, 4: 1.02, 5: 0.85, 6: 0.75, 7: 0.71, 8: 0.63, 9: 0.64, 10: 0.67, 11: 1.55, 12: 1.39, 13: 1.26, 14: 1.16, 15: 1.11, 16: 1.03, 17: 0.99, 18: 0.96, 19: 1.96, 20: 1.71, 21: 1.48, 22: 1.36, 23: 1.34, 24: 1.22, 25: 1.19, 26: 1.16, 27: 1.11, 28: 1.1, 29: 1.12, 30: 1.18, 31: 1.24, 32: 1.21, 33: 1.21, 34: 1.16, 35: 1.14, 36: 1.17, 37: 2.1, 38: 1.85, 39: 1.63, 40: 1.54, 41: 1.47, 42: 1.38, 43: 1.28, 44: 1.25, 45: 1.25, 46: 1.2, 47: 1.28, 48: 1.36, 49: 1.42, 50: 1.4, 51: 1.4, 52: 1.36, 53: 1.33, 54: 1.31, 55: 2.32, 56: 1.96, 57: 1.8, 58: 1.63, 59: 1.76, 60: 1.74, 62: 1.72, 63: 1.68, 64: 1.69, 65: 1.68, 66: 1.67, 67: 1.66, 68: 1.65, 69: 1.64, 70: 1.7, 71: 1.62, 72: 1.52, 73: 1.46, 74: 1.37, 75: 1.31, 76: 1.29, 77: 1.22, 78: 1.23, 79: 1.24, 80: 1.33, 81: 1.44, 82: 1.44, 83: 1.51}


In [59]:
z2FormationEnergy = {}
for z in ElemTable[['z']].values:
    try:
        z2FormationEnergy[z[0]] = element(z[0]).heat_of_formation
    except:
        print z[0]

In [60]:
print z2FormationEnergy

{1: 217.998, 2: None, 3: 159.3, 4: 324.0, 5: 565.0, 6: 716.87, 7: 472.44, 8: 249.229, 9: 79.335, 10: None, 11: 107.5, 12: 147.1, 13: 330.9, 14: 450.0, 15: 316.5, 16: 277.17, 17: 121.302, 18: None, 19: 89.0, 20: 177.8, 21: 377.8, 22: 473.0, 23: 515.5, 24: 397.48, 25: 283.3, 26: 415.5, 27: 426.7, 28: 430.1, 29: 337.4, 30: 130.4, 31: 271.96, 32: 372.0, 33: 302.5, 34: 227.2, 35: 111.85, 36: None, 37: 80.9, 38: 164.0, 39: 424.7, 40: 610.0, 41: 733.0, 42: 658.98, 43: 678.0, 44: 650.6, 45: 556.0, 46: 376.6, 47: 284.9, 48: 111.8, 49: 243.0, 50: 301.2, 51: 264.4, 52: 196.6, 53: 106.757, 54: None, 55: 76.5, 56: 179.1, 57: 431.0, 58: 420.1, 59: 356.9, 60: 326.9, 62: 206.7, 63: 177.4, 64: 397.5, 65: 388.7, 66: 290.4, 67: 300.6, 68: 316.4, 69: 232.2, 70: 155.6, 71: 427.6, 72: 618.4, 73: 782.0, 74: 851.0, 75: 774.0, 76: 787.0, 77: 669.0, 78: 565.7, 79: 368.2, 80: 61.38, 81: 182.2, 82: 195.2, 83: 209.6}


In [55]:
entry = WyckTable[120]
entry.to_dict().keys()

['Site symmetry', 'Multiplicity', 'Site generator', 'Wyckoff letter']

In [53]:
new_table = {}
for sg,entry in WyckTable.iteritems():
    new_table[sg] = entry.to_dict()

In [57]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [58]:
pp.pprint(new_table)

{   1: {   'Multiplicity': {   0: 1},
           'Site generator': {   0: ['x', 'y', 'z']},
           'Site symmetry': {   0: 1},
           'Wyckoff letter': {   0: 'a'}},
    2: {   'Multiplicity': {   0: 2,
                               1: 1,
                               2: 1,
                               3: 1,
                               4: 1,
                               5: 1,
                               6: 1,
                               7: 1,
                               8: 1},
           'Site generator': {   0: ['x', 'y', 'z'],
                                 1: [0.5, 0.5, 0.5],
                                 2: [0.0, 0.5, 0.5],
                                 3: [0.5, 0.0, 0.5],
                                 4: [0.5, 0.5, 0.0],
                                 5: [0.5, 0.0, 0.0],
                                 6: [0.0, 0.5, 0.0],
                                 7: [0.0, 0.0, 0.5],
                                 8: [0.0, 0.0, 0.0]},
           'Si

                                1: 4,
                                2: 4,
                                3: 4,
                                4: 4,
                                5: 4,
                                6: 4,
                                7: 2,
                                8: 2,
                                9: 2,
                                10: 2},
            'Site generator': {   0: ['x', 'y', 'z'],
                                  1: [0.0, 0.5, 'z'],
                                  2: [0.0, 0.0, 'z'],
                                  3: [0.5, 'y', 0.0],
                                  4: [0.0, 'y', 0.0],
                                  5: ['x', 0.0, 0.5],
                                  6: ['x', 0.0, 0.0],
                                  7: [0.0, 0.5, 0.0],
                                  8: [0.0, 0.0, 0.5],
                                  9: [0.5, 0.0, 0.0],
                                  10: [0.0, 0.0, 0.0]},
            'Site symm

    73: {   'Multiplicity': {   0: 16, 1: 8, 2: 8, 3: 8, 4: 8, 5: 8},
            'Site generator': {   0: ['x', 'y', 'z'],
                                  1: [0.0, 0.25, 'z'],
                                  2: [0.25, 'y', 0.0],
                                  3: ['x', 0.0, 0.25],
                                  4: [0.25, 0.25, 0.25],
                                  5: [0.0, 0.0, 0.0]},
            'Site symmetry': {   0: 1,
                                 1: '..2',
                                 2: '.2.',
                                 3: '2..',
                                 4: '-1',
                                 5: '-1'},
            'Wyckoff letter': {   0: 'f',
                                  1: 'e',
                                  2: 'd',
                                  3: 'c',
                                  4: 'b',
                                  5: 'a'}},
    74: {   'Multiplicity': {   0: 16,
                                1: 8,
               

                                   8: [0.0, 0.0, 0.25]},
             'Site symmetry': {   0: 1,
                                  1: '..2',
                                  2: '2..',
                                  3: '2..',
                                  4: '..2',
                                  5: '2.22',
                                  6: '-4..',
                                  7: '-4..',
                                  8: '2.22'},
             'Wyckoff letter': {   0: 'i',
                                   1: 'h',
                                   2: 'g',
                                   3: 'f',
                                   4: 'e',
                                   5: 'd',
                                   6: 'c',
                                   7: 'b',
                                   8: 'a'}},
    121: {   'Multiplicity': {   0: 16,
                                 1: 8,
                                 2: 8,
                                 3: 8,


                                 3: 6,
                                 4: 3,
                                 5: 3,
                                 6: 2,
                                 7: 2,
                                 8: 1,
                                 9: 1},
             'Site generator': {   0: ['x', 'y', 'z'],
                                   1: ['x', '-x', 'z'],
                                   2: ['x', 0.0, 0.5],
                                   3: ['x', 0.0, 0.0],
                                   4: [0.5, 0.0, 0.5],
                                   5: [0.5, 0.0, 0.0],
                                   6: [   0.3333333333333333,
                                          0.6666666666666666,
                                          'z'],
                                   7: [0.0, 0.0, 'z'],
                                   8: [0.0, 0.0, 0.5],
                                   9: [0.0, 0.0, 0.0]},
             'Site symmetry': {   0: 1,
                 

                                   3: ['x', 0.5, 0.0],
                                   4: ['x', 'x', 'x'],
                                   5: ['x', 0.5, 0.5],
                                   6: ['x', 0.0, 0.0],
                                   7: [0.5, 0.0, 0.0],
                                   8: [0.0, 0.5, 0.5],
                                   9: [0.5, 0.5, 0.5],
                                   10: [0.0, 0.0, 0.0]},
             'Site symmetry': {   0: 1,
                                  1: '..2',
                                  2: '..2',
                                  3: '2..',
                                  4: '.3.',
                                  5: '4..',
                                  6: '4..',
                                  7: '42.2',
                                  8: '42.2',
                                  9: '432',
                                  10: '432'},
             'Wyckoff letter': {   0: 'k',
                               

# make input

In [28]:
print WyckTable[120]
len(WyckTable[120])

   Multiplicity Wyckoff letter Site symmetry    Site generator
0            16              i             1         [x, y, z]
1             8              h           ..2   [x, x+0.5, 0.0]
2             8              g           2..     [0.0, 0.5, z]
3             8              f           2..     [0.0, 0.0, z]
4             8              e           ..2      [x, x, 0.25]
5             4              d          2.22   [0.0, 0.5, 0.0]
6             4              c          -4..  [0.0, 0.5, 0.25]
7             4              b          -4..   [0.0, 0.0, 0.0]
8             4              a          2.22  [0.0, 0.0, 0.25]


9

## make the initial srtucture

In [99]:
cc = crystal(symbols=['Si'], basis=[0.1,0.4,0.6], spacegroup=86, cellpar=[1,1,1,90,90,90],symprec=1e-5, pbc=True)
print cc.get_scaled_positions()

[[ 0.1  0.4  0.6]
 [ 0.9  0.6  0.6]
 [ 0.1  0.6  0.1]
 [ 0.9  0.4  0.1]
 [ 0.4  0.1  0.9]
 [ 0.6  0.9  0.9]
 [ 0.4  0.9  0.4]
 [ 0.6  0.1  0.4]]


In [97]:
seed = 53485
vdw_ratio = 1.5
sites_z = [14]

out = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
print out
print out[0].get_scaled_positions()

0.745 0.646 0.425
(Atoms(symbols='Si2', pbc=True, cell=[0.999701798109813, 7.282, 7.105]), 26, ['b'])
[[ 0.5    0.646  0.425]
 [ 0.5    0.354  0.925]]


## unskew cell

In [42]:
seed = 86
vdw_ratio = 1.5
sites_z = [14]
sg = 191
cc = input2crystal(sites_z,seed,vdw_ratio,WyckTable,sg)[0]

frame = cc.copy()
lattice = frame.get_cell()
lengths = frame.get_cell_lengths_and_angles()[:3]
angles = np.cos(frame.get_cell_lengths_and_angles()[3:]*d2r)

max_angle2ij = {0:(0,1),1:(0,2),2:(1,2)}
for max_angle in np.where(np.abs(angles)>0.5)[0]:
    
    i,j = max_angle2ij[max_angle]
    if lengths[i] > lengths[j]:
        i,j = j,i 
    lattice[j,:] = lattice[j,:] - np.round(angles[max_angle]) * lattice[i,:]
    
frame.set_cell(lattice)
frame.wrap()

print compare(cc,frame)

1.0


In [41]:
view(cc)

In [43]:
frame = unskewCell(cc)
view([frame,cc])

## LJ

In [25]:
seed = 1
vdw_ratio = 1.5
sites_z = [14]
# sg = 191
ff = 2**(1./6.)
r = z2Covalentradius[14]
sigma = 2*r/ff
r_c = 3.*sigma
print sigma,r_c

2.06688502609 6.20065507826


In [27]:
crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio)
pr = 1e-2
pressure = np.eye(3)*pr

param_str,cutoff_max = make_LJ_input(sites_z)
cc_qp = ase2qp(crystal)
pot = Potential('IP LJ',param_str=param_str)
cc_qp.set_calculator(pot)

cc_qp.get_stress()
# minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile=None,method='fire',
#                   external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=True)
# minimiser.run(fmax=1e-5,steps=1e6)



# view([crystal,cc_qp])


0.676 0.1 0.342


array([ -2.42259900e+04,   4.91498744e-03,   6.31512357e-03,
         4.19232592e-21,  -0.00000000e+00,  -4.19232592e-21])

In [67]:
%%time
crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
pr = 1e-0
pressure = np.eye(3)*pr

param_str = make_LJ_input(sites_z)
cc_qp = ase2qp(crystal)
pot = Potential('IP LJ',param_str=param_str)
cc_qp.set_calculator(pot)
cc_qp = LJrelax(cc_qp,FIRE,optimizerOptions={'dtmax':10.,'dt':0.1,'finc':1.1, 
                                             'fdec':0.5, 'astart':0.1, 'fa':0.99, 'a':0.1},
                runOptions={'fmax':1e-5,'steps':1e4})
cc_qp = ase2qp(cc_qp)
#view([crystal,cc_qp])

CPU times: user 15.3 s, sys: 524 ms, total: 15.9 s
Wall time: 15.9 s


In [ ]:
%%time
for seed in range(100):
    crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
    pr = 1e-0
    pressure = np.eye(3)*pr

    param_str = make_LJ_input(sites_z)
    cc_qp = ase2qp(crystal)
    pot = Potential('IP LJ',param_str=param_str)
    cc_qp.set_calculator(pot)


    minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)
    minimiser.run(fmax=1e-5,steps=1e6)

In [16]:
Minim?

In [ ]:
dt=0.1, maxmove=0.2, dtmax=1.0, Nmin=5, finc=1.1, fdec=0.5, astart=0.1, fa=0.99, a=0.1,

In [ ]:
eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0, external_pressure=None, use_precond=0

In [32]:
crystal,sg,wki = input2crystal([14],14,1.5,WyckTable)
print sg,wki
view(crystal)

108 ['d']


In [18]:
crystal

Atoms(symbols='Si', pbc=True, cell=[[nan, nan, nan], [-1.3442709097535988, 3.9040461986495787, 0.0], [nan, nan, nan]])

In [33]:
aa = ase2qp(crystal)
print aa.get_all_distances().min()

0.0


In [34]:
crystal,sg,wki = input2crystal([14],14,1.5,WyckTable)

cc_qp = ase2qp(crystal)

Natom = cc_qp.get_number_of_atoms()
print (cc_qp.get_all_distances() + np.eye(Natom)).min()
aa.rattle(1e-10)
print (cc_qp.get_all_distances() + np.eye(Natom)).min()

0.162183963368
0.162183963193


In [33]:
seed = 5348
vdw_ratio = 1.5
sites_z = [14]

crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio,WyckTable)
pr = 1e-2
pressure = np.eye(3)*pr

param_str,max_cutoff = make_LJ_input(sites_z)
print param_str
cc_qp = ase2qp(unskewCell(crystal))
pot = Potential('IP LJ',param_str=param_str)
cc_qp.set_calculator(pot)
cc_qp.set_cutoff(max_cutoff,0.5)
Natom = cc_qp.get_number_of_atoms()

dyn = FIRE(cc_qp, logfile=None)
mf = np.linalg.norm(cc_qp.get_forces(), axis=1).max()
while mf > 1e5:
    dyn.run(**{'fmax': 3e-2, 'steps': 1})
    mf = np.linalg.norm(cc_qp.get_forces(), axis=1).max()

minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                  external_pressure=pressure,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

minimiser.run(fmax=1e-2,steps=1e6)

minimiser = Minim(cc_qp, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                  external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

minimiser.run(fmax=1e-6,steps=1e6)

view([crystal,cc_qp])

<LJ_params n_types="1" label="default"> <per_type_data type="1" atomic_num="14" /> <per_pair_data type1="1" type2="1" sigma="2.06688502609" eps6="1.0" eps12="1.0" cutoff="6.20065507826" energy_shift="F" linear_force_shift="F" /> </LJ_params>


In [45]:
def atoms2np(crystal):
    lattice = crystal.get_cell()
    positions = crystal.get_positions()
    numbers = crystal.get_atomic_numbers()
    return numbers,positions,lattice
def np2atoms(numbers,positions,lattice,type='ase'):
    if type == 'ase':
        from ase import Atoms as aseAtoms
        crystal = aseAtoms(numbers=numbers,cell=lattice,positions=positions)
    elif type == 'quippy':
        from quippy import Atoms as qpAtoms
        crystal = qpAtoms(numbers=numbers,cell=lattice,positions=positions)
    else:
        raise TypeError('Not recognised input type {}'.format(type))
    return crystal

In [46]:
def LJ_vcrelax(sites_z,numbers,positions,lattice,isotropic_external_pressure=1e-2):
    from quippy.potential import Potential, Minim
    from ase.optimize import  FIRE
    import numpy as np
    
    crystal = np2atoms(numbers,positions,lattice,type='quippy')
    
    pressure_tensor = np.eye(3)*isotropic_external_pressure
    # get the string to setup the quippy LJ potential (parameters and species)
    param_str,max_cutoff = make_LJ_input(sites_z)
    
    pot = Potential('IP LJ',param_str=param_str)
    crystal.set_calculator(pot)
    crystal.set_cutoff(max_cutoff,0.5)
    
    # use ASE fire implementation to relax the internal d.o.g. to make sure atoms are not too close to each other 
    # when optimizing with quippy's fire implemnetation (it crashes otherwise)
    dyn = FIRE(crystal, logfile=None)
    max_force = np.linalg.norm(crystal.get_forces(), axis=1).max()
    # the threshold here make sure that quippy will not exit with out of memory error
    while max_force > 1e5:
        dyn.run(**{'fmax': 3e-2, 'steps': 1})
        max_force = np.linalg.norm(crystal.get_forces(), axis=1).max()
    
    # 1st round of vc relax with external isotropic pressure
    minimiser = Minim(crystal, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=pressure_tensor,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

    minimiser.run(fmax=5e-3,steps=1e6)
    
    # 2nd round of vc relax without external isotropic pressure
    minimiser = Minim(crystal, relax_positions=True, relax_cell=True,logfile='-',method='fire',
                      external_pressure=None,eps_guess=0.2, fire_dt0=0.1, fire_dt_max=1.0,use_precond=None)

    minimiser.run(fmax=1e-6,steps=1e6)
    
    crystal.wrap()
    
    numbers,positions,lattice = atoms2np(crystal)
    
    return numbers,positions,lattice

# general scheme

In [73]:
seed = 99
vdw_ratio = 1.5
sites_z = [14]

crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio)


In [74]:
crystal = unskewCell(crystal)

crystal = LJ_vcrelax(sites_z,crystal,isotropic_external_pressure=1e-2)

crystal = get_standard_frame(crystal,to_primitive=False,symprec=1e-5)

In [75]:
crystal = LJ_vcrelax(sites_z,crystal,isotropic_external_pressure=1e-2)

In [80]:
crystal = get_standard_frame(crystal,to_primitive=False,symprec=1e-5)

In [81]:
data = spg.get_symmetry_dataset(crystal,symprec=1e-5)
print data.keys()
print sg,wki
print data['number'],data['wyckoffs'],data['equivalent_atoms']

['std_positions', 'equivalent_atoms', 'std_types', 'translations', 'rotations', 'number', 'choice', 'transformation_matrix', 'std_mapping_to_primitive', 'origin_shift', 'hall_number', 'mapping_to_primitive', 'pointgroup', 'international', 'wyckoffs', 'hall', 'std_lattice']
229 ['d']
229 ['d', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'd'] [0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
%%time
vdw_ratio = 1.5
sites_z = [14]

inputs = [[sites_z,seed,vdw_ratio] for seed in range(100)]
res = []
for inp in tqdm_notebook(inputs):
    res.append(generate_crystal_step_1_wrapper(inp))


HBox(children=(IntProgress(value=0), HTML(value=u'')))

0.775 0.786 0.778


In [ ]:
view(res[100])

## test special cases

In [18]:
sites_z,seed,vdw_ratio = [[14], 1, 1.5]
crystal,sg,wki = input2crystal(sites_z,seed,vdw_ratio)
view(crystal)
crystal = unskewCell(crystal)
view(crystal)

0.676 0.1 0.342


In [8]:
print crystal.get_cell_lengths_and_angles()

[   0.55899479    9.78          6.03389907   98.83400165  125.93040169
   94.        ]


In [9]:
sg,wki

(1, ['a'])

In [22]:
crystal = unskewCell(crystal)

crystal = LJ_vcrelax(sites_z,crystal,isotropic_external_pressure=1e-2)

In [19]:
Minim?